In [1]:
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error  
import matplotlib.pyplot as plt


In [1]:
import pandas as pd
x0 = pd.read_excel(r'nanoplastic_std_x0.xlsx')
y = pd.read_excel(r'nanoplastic_std_y.xlsx')


In [32]:


Xtrain,Xtest,Ytrain,Ytest = train_test_split(x0,y,test_size=0.2,random_state=170)

'''

Parameter optimization and five-fold cross validation and parameter optimization

'''
#Random_state
score_5cv_all = []
for i in range(0, 400, 1):
    rfc =RandomForestRegressor(random_state=i)
    score_5cv =cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(score_5cv_all.index(score_max_5cv)))

random_state_5cv = range(0, 400)[score_5cv_all.index(max(score_5cv_all))]
print(random_state_5cv)



#n_estimator
score_5cv_all = []
for i in range(1, 500, 1):
    rfc = RandomForestRegressor(n_estimators=i
                                , random_state=random_state_5cv)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,500)[score_5cv_all.index(score_max_5cv)]
print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))



#Max_depth
score_5cv_all = []
for i in range(1, 200, 1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                , random_state= random_state_5cv 
                                , max_depth=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
max_depth_5cv = range(1,200)[score_5cv_all.index(score_max_5cv)]
print(max_depth_5cv )


#min_samples_leaf 
score_5cv_all = []
for i in range(1, 100, 1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                , random_state= random_state_5cv 
                                , max_depth=max_depth_5cv
                                ,min_samples_leaf=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "min_samples_leaf_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
min_samples_leaf_5cv = range(1,200)[score_5cv_all.index(score_max_5cv)]
print(min_samples_leaf_5cv )

#min_samples_split
score_5cv_all = []
for i in range(1, 100, 1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                , random_state= random_state_5cv 
                                , max_depth=max_depth_5cv
                                ,min_samples_leaf=min_samples_leaf_5cv
                                ,min_samples_split=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "min_samples_split_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
min_samples_split_5cv = range(1,200)[score_5cv_all.index(score_max_5cv)]
print(min_samples_split_5cv )




#max_features
score_5cv_all = []
for i in range(1,Xtrain.shape[1]+1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=max_depth_5cv
                                ,min_samples_leaf=min_samples_leaf_5cv
                                ,min_samples_split=min_samples_split_5cv
                                ,max_features=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_features_5cv = range(1, Xtrain.shape[1]+1)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_features_5cv:{}".format(max_features_5cv))




rfc = RandomForestRegressor(n_estimators=n_est_5cv,
                            random_state= random_state_5cv,
                            max_depth=max_depth_5cv,
                            max_features=max_features_5cv,
                            min_samples_leaf=min_samples_leaf_5cv,
                            min_samples_split = min_samples_split_5cv)
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
mae = mean_absolute_error(Ytrain,CV_predictions)
print("5cv:",CV_score)
print("rmse_5CV",rmse)
print('5CV_MAE:',mae)
expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions}
pd.DataFrame(expvspred_5cv).to_excel(r'random_forest_5fcv_predictions.xlsx')

'''
Test set validation

'''


test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
rmse = np.sqrt(mean_squared_error(Ytest,test_predictions))
mae = mean_absolute_error(Ytest,test_predictions)
print("test:",score_test)
print("rmse_test",rmse)
print("MAE_test:",mae)
expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
pd.DataFrame(expvspred_test).to_excel(r'RF_test_predictions.xlsx')




Best_5cv score：0.739963755212947 random_5cv:233
233
Best_5cv score：0.7411816274182756 n_est_5cv:88
Best_5cv score：0.741320970446487 max_depth_5cv:44
45
Best_5cv score：0.741320970446487 min_samples_leaf_5cv:0
1
Best_5cv score：0.741320970446487 min_samples_split_5cv:0
1
Best_5cv score：0.7427457405932671 max_features_5cv:120
5cv: 0.7427457405932671
rmse_5CV 12.160378829514203
5CV_MAE: 7.81646765379725
test: 0.7620316182923639
rmse_test 10.670822699745228
MAE_test: 7.072492299680643


In [4]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(x0,y,test_size=0.2,random_state=170)

In [7]:

rfc = RandomForestRegressor(n_estimators=88,
                            random_state= 233,
                            max_depth=44,
                            max_features=120,
                            min_samples_leaf=1,
                            min_samples_split =1)

In [8]:
regressor = rfc.fit(Xtrain, Ytrain)

<ipython-input-8-861de99e5462>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor = rfc.fit(Xtrain, Ytrain)


In [9]:
#feature importance
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
feature_importance = regressor.feature_importances_
dataframe_feature_importance = sorted(zip(feature_importance,Xtrain.columns),reverse=True)
df = pd.DataFrame(dataframe_feature_importance,columns=["importance","feature"])
# 提取特征名称中的前缀，作为新的一列
df["prefix"] = df["feature"].str.split("_").str[0]
df_new = df.groupby("prefix")["importance"].sum().reset_index()
df_new.sort_values(by='importance',ascending=False,inplace=True)



In [11]:
df_new.to_excel(r'f_importance.xlsx')